# Library

In [ ]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# Import Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define root directory
dir_sehat = '/content/drive/MyDrive/new dataset/sehat'
dir_sakit = '/content/drive/MyDrive/new dataset/sakit'

# Split Training Validation Test




Untuk membuat Directory Training dan Validation

kalau udh displit di drive, gausah displit lagi

In [ ]:
def split_data(root_dir, train_size=0.95, val_size=0.1, test_size=0.05, random_seed=42):
    # Membuat direktori untuk train, val, dan test
    train_dir = os.path.join(root_dir, 'train')
    val_dir = os.path.join(root_dir, 'val')
    test_dir = os.path.join(root_dir, 'test')

    # Membuat direktori train jika belum ada
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)

    # Membuat direktori val jika belum ada
    if not os.path.exists(val_dir):
        os.makedirs(val_dir)

    # Membuat direktori test jika belum ada
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)

    # Loop melalui setiap direktori (jenis tanaman)
    for plant_type in os.listdir(root_dir):
        if os.path.isdir(os.path.join(root_dir, plant_type)):
            # Membuat sub-direktori untuk setiap jenis tanaman di setiap direktori (train, val, test)
            train_plant_dir = os.path.join(train_dir, plant_type)
            val_plant_dir = os.path.join(val_dir, plant_type)
            test_plant_dir = os.path.join(test_dir, plant_type)

            if not os.path.exists(train_plant_dir):
                os.makedirs(train_plant_dir)

            if not os.path.exists(val_plant_dir):
                os.makedirs(val_plant_dir)

            if not os.path.exists(test_plant_dir):
                os.makedirs(test_plant_dir)

            # Mengambil daftar file gambar untuk setiap jenis tanaman
            plant_images = [img for img in os.listdir(os.path.join(root_dir, plant_type)) if os.path.isfile(os.path.join(root_dir, plant_type, img))]

            # Membagi data menjadi train, val, dan test
            train_images, test_images = train_test_split(plant_images, test_size=val_size + test_size, random_state=random_seed)
            val_images, test_images = train_test_split(test_images, test_size=test_size/(val_size + test_size), random_state=random_seed)

            # Memindahkan gambar ke dalam direktori yang sesuai
            for image in train_images:
                src_path = os.path.join(root_dir, plant_type, image)
                dest_path = os.path.join(train_plant_dir, image)
                shutil.copy(src_path, dest_path)

            for image in val_images:
                src_path = os.path.join(root_dir, plant_type, image)
                dest_path = os.path.join(val_plant_dir, image)
                shutil.copy(src_path, dest_path)

            for image in test_images:
                src_path = os.path.join(root_dir, plant_type, image)
                dest_path = os.path.join(test_plant_dir, image)
                shutil.copy(src_path, dest_path)

In [ ]:
try:
    split_sehat = split_data(dir_sehat)
    if len(split_sehat) == 0:
        raise ValueError("No data found in the directory.")
    else:
        print("Plant Label Map:", split_sehat)
    # Continue with the rest of your code
except FileExistsError:
    print("You should not be seeing this since the upper directory is removed beforehand")
except ValueError as ve:
    print(ve)
    # Handle the case when no data is found

NameError: ignored

In [ ]:
label_map = split_data(dir_sakit)
print("Plant Label Map:", label_map)


ValueError: ignored

# Cek direktori

In [ ]:
def cek_dir(root_dir):
  ada_sub_direktori_kosong = False  # Inisialisasi flag
  for rootdir, dirs, files in os.walk(root_dir):
      for subdir in reversed(dirs):
          subdir_path = os.path.join(rootdir, subdir)
          if len(os.listdir(subdir_path)) == 0:
              print(f"Sub-direktori kosong: {subdir_path}")
              dirs.remove(subdir)
              os.rmdir(subdir_path)
          else:
                print(f"Sub-direktori berisi file: {subdir_path}")
  # Cek flag dan cetak pesan sesuai
  if not ada_sub_direktori_kosong:
    print("Tidak ada sub-direktori kosong.")

In [ ]:
cek_dir(dir_sehat)
print("\n")
cek_dir(dir_sakit)


Sub-direktori berisi file: /content/drive/MyDrive/new dataset/sehat/test
Sub-direktori berisi file: /content/drive/MyDrive/new dataset/sehat/val
Sub-direktori berisi file: /content/drive/MyDrive/new dataset/sehat/train
Sub-direktori berisi file: /content/drive/MyDrive/new dataset/sehat/kol
Sub-direktori berisi file: /content/drive/MyDrive/new dataset/sehat/selada
Sub-direktori berisi file: /content/drive/MyDrive/new dataset/sehat/daun timun
Sub-direktori berisi file: /content/drive/MyDrive/new dataset/sehat/timun
Sub-direktori berisi file: /content/drive/MyDrive/new dataset/sehat/sawi
Sub-direktori berisi file: /content/drive/MyDrive/new dataset/sehat/bok choy
Sub-direktori berisi file: /content/drive/MyDrive/new dataset/sehat/tomato
Sub-direktori berisi file: /content/drive/MyDrive/new dataset/sehat/daun tomat
Sub-direktori berisi file: /content/drive/MyDrive/new dataset/sehat/train/kol
Sub-direktori berisi file: /content/drive/MyDrive/new dataset/sehat/train/selada
Sub-direktori beri

In [ ]:

# Loop melalui setiap folder train dan tampilkan jumlah data
def cek_train(train_dir):
  for plant_folder in os.listdir(train_dir):
      plant_path = os.path.join(train_dir, plant_folder)

      if os.path.isdir(plant_path):
          num_train_data = len(os.listdir(plant_path))
          print(f"Train data for {plant_folder}: {num_train_data} images")

# Loop melalui setiap folder validation dan tampilkan jumlah data
def cek_val(val_dir):
  for plant_folder in os.listdir(val_dir):
      plant_path = os.path.join(val_dir, plant_folder)

      if os.path.isdir(plant_path):
          num_val_data = len(os.listdir(plant_path))
          print(f"Validation data for {plant_folder}: {num_val_data} images")

# Loop melalui setiap folder test dan tampilkan jumlah data
def cek_val(val_dir):
  for plant_folder in os.listdir(val_dir):
      plant_path = os.path.join(val_dir, plant_folder)

      if os.path.isdir(plant_path):
          num_val_data = len(os.listdir(plant_path))
          print(f"test data for {plant_folder}: {num_val_data} images")

In [ ]:
train_sehat = '/content/drive/MyDrive/new dataset/sehat/train'
val_sehat = '/content/drive/MyDrive/new dataset/sehat/val'
test_sehat = '/content/drive/MyDrive/new dataset/sehat/test'
train_sakit = '/content/drive/MyDrive/new dataset/sakit/train'
val_sakit = '/content/drive/MyDrive/new dataset/sakit/val'
test_sakit = '/content/drive/MyDrive/new dataset/sakit/test'

cek_train(train_sehat)
print("\n")
cek_val(val_sehat)
print("\n")
cek_val(test_sehat)
print("\n\n")
cek_train(train_sakit)
print("\n")
cek_val(val_sakit)
print("\n")
cek_val(test_sakit)


Train data for daun tomat: 181 images
Train data for tomato: 157 images
Train data for bok choy: 173 images
Train data for sawi: 196 images
Train data for timun: 207 images
Train data for daun timun: 192 images
Train data for selada: 174 images
Train data for kol: 188 images


test data for daun tomat: 22 images
test data for tomato: 24 images
test data for bok choy: 20 images
test data for sawi: 23 images
test data for timun: 25 images
test data for daun timun: 22 images
test data for selada: 21 images
test data for kol: 22 images


test data for daun tomat: 14 images
test data for tomato: 17 images
test data for bok choy: 14 images
test data for sawi: 16 images
test data for timun: 20 images
test data for daun timun: 19 images
test data for selada: 16 images
test data for kol: 16 images



Train data for unhealhty fruit: 204 images
Train data for unhealthy leaf: 316 images


test data for unhealhty fruit: 27 images
test data for unhealthy leaf: 40 images


test data for unhealhty fru

# Image Data Generator

**training datagen untuk tanaman sehat**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Tentukan path direktori data tanaman dan penyakit
train_dir_plant = '/content/drive/MyDrive/new dataset/sehat/train'
train_dir_disease = '/content/drive/MyDrive/new dataset/sakit/train'
validation_dir_plant = '/content/drive/MyDrive/new dataset/sehat/val'
validation_dir_disease = '/content/drive/MyDrive/new dataset/sakit/val'
test_dir_plant = '/content/drive/MyDrive/new dataset/sehat/test'
test_dir_disease = '/content/drive/MyDrive/new dataset/sakit/test'

# Inisialisasi ImageDataGenerator untuk augmentasi dan normalisasi data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Gunakan flow_from_directory untuk memuat data dari direktori
train_generator_plant = train_datagen.flow_from_directory(
    train_dir_plant,
    target_size=(150, 150, 3),
    batch_size=32,
    class_mode='categorical'  # Sesuaikan dengan tipe klasifikasi yang Anda lakukan
)
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator_plant = validation_datagen.flow_from_directory(
    validation_dir_plant,
    target_size=(150, 150, 3),
    batch_size=32,
    class_mode='categorical'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator_plant = test_datagen.flow_from_directory(
    test_dir_plant,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

train_generator_disease = train_datagen.flow_from_directory(
    train_dir_disease,
    target_size=(150, 150, 3),
    batch_size=32,
    class_mode='categorical'
)

validation_generator_disease = validation_datagen.flow_from_directory(
    validation_dir_disease,
    target_size=(150, 150, 3),
    batch_size=32,
    class_mode='categorical'
)
test_generator_disease = test_datagen.flow_from_directory(
    test_dir_disease,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 1468 images belonging to 8 classes.
Found 179 images belonging to 8 classes.
Found 132 images belonging to 8 classes.
Found 520 images belonging to 2 classes.
Found 67 images belonging to 2 classes.
Found 70 images belonging to 2 classes.


**training datagen untuk tanaman sakit**

**DEFINE MODEL**

# Define Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [ ]:
def model_jenis_tanaman():
  model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(8, activation='softmax')
  ])
  model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
# Print the model summary
model_tanaman = model_jenis_tanaman()
model_tanaman.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 148, 148, 128)     3584      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 74, 74, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 72, 72, 64)        73792     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 34, 34, 32)        18464     
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 17, 17, 32)       

In [ ]:
from tensorflow.keras import layers, models

# Assuming output_dim_model1 is the number of features in the output of model_tanaman
output_dim_model1 = model_tanaman.output_shape[1]

In [ ]:
def model_deteksi_penyakit():
  model = tf.keras.models.Sequential([
    # This is the first convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=(output_dim_model1 , output_dim_model1 , 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The 4 convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
  ])
  model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

# Print the model summary
model_penyakit = model_deteksi_penyakit()
model_penyakit.summary()

ValueError: ignored

# training Model

In [ ]:
class Callback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
      # Check the loss
      if(logs.get('loss') < 0.5 or logs.get('accuracy') > 0.8):

        # Stop if threshold is met
        print("\nLoss is lower than 0.4 and acc is above 0.8 so cancelling training!")
        self.model.stop_training = True
        print(f'Epoch {epoch} ended with logs: {logs}')
# Instantiate class
callbacks = Callback()

In [ ]:
# train tanaman
model_tanaman.fit(train_generator_plant,
                    epochs=20,
                    verbose=1,
                    validation_data=validation_generator_plant,
                    callbacks=[callbacks])

In [ ]:
# train tanaman sakit
model_penyakit.fit(train_generator_disease,
                    epochs=20,
                    verbose=1,
                    validation_data=validation_generator_disease,
                    callbacks=[callbacks])

In [ ]:
nama_tanaman = ['bok choy', 'daun timun', 'daun tomat', 'kol', 'sawi', 'selada', 'timun', 'tomat']
# Assuming `penyakit` is your first model and `model_penyakit` is the second model
# Assuming `train_generator_sakit` and `validation_generator_sakit` are your data generators

# Train the first model
model_tanaman.fit(train_generator_plant, epochs=20, verbose=1, validation_data=validation_generator_plant, callbacks=[callbacks])

# Train the second model
model_penyakit.fit(train_generator_disease, epochs=20, verbose=1, validation_data=validation_generator_disease, callbacks=[callbacks])

# Get predictions from the two models
predictions_penyakit_train = model_penyakit.predict(train_generator_plant)
predictions_model_penyakit_train = model_penyakit.predict(train_generator_disease)

# Stack the predictions horizontally
stacked_predictions_train = np.hstack((predictions_penyakit_train, predictions_model_penyakit_train))

# Create a logistic regression model for stacking
stacking_model = StackingClassifier(estimators=[('tanaman', model_tanaman), ('penyakit', model_penyakit)],
                                    final_estimator=LogisticRegression())

# Train the stacking model on the stacked predictions
stacking_model.fit(stacked_predictions_train, target_variable_for_training)

# Now you can use the stacking model for predictions on new data
# Repeat the process for validation data and test data if needed


# Predict Model tanaman

**Rename file agar mudah terbaca**

In [ ]:
def rename(directory_):
  for folder_name in os.listdir(directory_):
      # Mendapatkan path lengkap folder
      folder_path = os.path.join(directory_, folder_name)

      # Memeriksa apakah itu benar-benar folder
      if os.path.isdir(folder_path):
          # Mendapatkan daftar file dalam folder
          files = os.listdir(folder_path)

          # Mengurutkan file
          files.sort()

          # Mengganti nama file sesuai dengan nama folder dan urutan
          for i, file_name in enumerate(files):
              # Membentuk nama baru
              new_name = f"{folder_name}_{i + 1}{os.path.splitext(file_name)[1]}"

              # Path file lama dan baru
              old_path = os.path.join(folder_path, file_name)
              new_path = os.path.join(folder_path, new_name)

              # Mengganti nama file
              os.rename(old_path, new_path)

In [ ]:
rename('/content/drive/MyDrive/new dataset/sehat/test')
rename('/content/drive/MyDrive/new dataset/sakit/test')


In [ ]:
sehat_test_dir ='/content/drive/MyDrive/new dataset/sehat/test'
# Buat objek ImageDataGenerator untuk memuat gambar dari direktori
test_datagen_sehat = ImageDataGenerator(rescale=1./255)
test_generator_sehat = test_datagen_sehat.flow_from_directory(
    sehat_test_dir,
    target_size=(150, 150),
    batch_size=1,
    class_mode='categorical',  # Sesuaikan dengan jenis kelas data Anda
    shuffle=False  # Pastikan tidak ada pengacakan data
  )

sakit_test_dir ='/content/drive/MyDrive/new dataset/sakit/test'
# Buat objek ImageDataGenerator untuk memuat gambar dari direktori
test_datagen_sakit = ImageDataGenerator(rescale=1./255)
test_generator_sakit = test_datagen_sakit.flow_from_directory(
    sakit_test_dir,
    target_size=(150, 150),
    batch_size=1,
    class_mode='categorical',  # Sesuaikan dengan jenis kelas data Anda
    shuffle=False  # Pastikan tidak ada pengacakan data
  )


Found 39 images belonging to 8 classes.
Found 35 images belonging to 2 classes.


In [ ]:
# Lakukan prediksi untuk setiap gambar dalam direktori test
predictions = jenis_tanaman.predict(test_generator_sehat)
nama_tanaman = ['bok choy', 'daun timun', 'daun tomat', 'kol', 'sawi', 'selada', 'timun', 'tomat']
results = []
for i, prediction in enumerate(predictions):
    # Ambil nama file gambar
    image_filename = os.path.basename(test_generator_sehat.filenames[i])

    # Ambil indeks hasil prediksi
    predicted_index = np.argmax(prediction)

    # Ganti hasil prediksi dengan nama buah sesuai indeks
    predicted_label = nama_tanaman[predicted_index]

    # Simpan pasangan (nama file, hasil prediksi) dalam list
    results.append((image_filename, predicted_label))

# Urutkan berdasarkan nama file
results.sort(key=lambda x: x[0])

# Tampilkan hasil prediksi
for i, (image_filename, predicted_label) in enumerate(results):
    # Dapatkan path lengkap untuk gambar
    image_path = os.path.join(sehat_test_dir, image_filename)

    # Tampilkan hasil prediksi
    print(f"Gambar {i + 1} - nama file: {image_filename}, Prediksi: {predicted_label}")

39/39 [==============================] - 2s 39ms/step
Gambar 1 - nama file: bok choy_1.jpg, Prediksi: selada
Gambar 2 - nama file: bok choy_2.jpg, Prediksi: bok choy
Gambar 3 - nama file: bok choy_3.jpg, Prediksi: timun
Gambar 4 - nama file: daun timun_1.jpg, Prediksi: daun timun
Gambar 5 - nama file: daun timun_2.jpg, Prediksi: sawi
Gambar 6 - nama file: daun timun_3.jpg, Prediksi: daun timun
Gambar 7 - nama file: daun timun_4.jpg, Prediksi: daun timun
Gambar 8 - nama file: daun timun_5.JPG, Prediksi: kol
Gambar 9 - nama file: daun timun_6.JPG, Prediksi: daun timun
Gambar 10 - nama file: daun timun_7.JPG, Prediksi: kol
Gambar 11 - nama file: daun tomat_1.JPG, Prediksi: daun tomat
Gambar 12 - nama file: daun tomat_2.JPG, Prediksi: daun tomat
Gambar 13 - nama file: daun tomat_3.JPG, Prediksi: daun tomat
Gambar 14 - nama file: kol_1.jpg, Prediksi: kol
Gambar 15 - nama file: kol_2.jpg, Prediksi: kol
Gambar 16 - nama file: kol_3.jpg, Prediksi: kol
Gambar 17 - nama file: kol_4.jpg, Prediksi

In [ ]:
# Lakukan prediksi untuk setiap gambar dalam direktori test
predictions = penyakit.predict(test_generator_sakit)
jenis_penyakit = ['buah busuk', 'daun rusak']
results = []
for i, prediction in enumerate(predictions):
    # Ambil nama file gambar
    image_filename = os.path.basename(test_generator_sakit.filenames[i])

    # Ambil indeks hasil prediksi
    predicted_index = np.argmax(prediction)

    # Ganti hasil prediksi dengan nama buah sesuai indeks
    predicted_label = jenis_penyakit[predicted_index]

    # Simpan pasangan (nama file, hasil prediksi) dalam list
    results.append((image_filename, predicted_label))

# Urutkan berdasarkan nama file
results.sort(key=lambda x: x[0])

# Tampilkan hasil prediksi
for i, (image_filename, predicted_label) in enumerate(results):
    # Dapatkan path lengkap untuk gambar
    image_path = os.path.join(sakit_test_dir, image_filename)

    # Tampilkan hasil prediksi
    print(f"Gambar {i + 1} - nama file: {image_filename}, Prediksi: {predicted_label}")

35/35 [==============================] - 2s 47ms/step
Gambar 1 - nama file: unhealhty fruit_1.jpg, Prediksi: buah busuk
Gambar 2 - nama file: unhealhty fruit_10.jpg, Prediksi: daun rusak
Gambar 3 - nama file: unhealhty fruit_11.jpg, Prediksi: buah busuk
Gambar 4 - nama file: unhealhty fruit_12.jpg, Prediksi: buah busuk
Gambar 5 - nama file: unhealhty fruit_13.jpg, Prediksi: buah busuk
Gambar 6 - nama file: unhealhty fruit_14.jpg, Prediksi: buah busuk
Gambar 7 - nama file: unhealhty fruit_2.jpg, Prediksi: buah busuk
Gambar 8 - nama file: unhealhty fruit_3.jpg, Prediksi: buah busuk
Gambar 9 - nama file: unhealhty fruit_4.jpg, Prediksi: daun rusak
Gambar 10 - nama file: unhealhty fruit_5.jpg, Prediksi: buah busuk
Gambar 11 - nama file: unhealhty fruit_6.jpg, Prediksi: buah busuk
Gambar 12 - nama file: unhealhty fruit_7.jpg, Prediksi: buah busuk
Gambar 13 - nama file: unhealhty fruit_8.jpg, Prediksi: buah busuk
Gambar 14 - nama file: unhealhty fruit_9.jpg, Prediksi: buah busuk
Gambar 15 - 

# **Predict gabungan**

In [ ]:
# Lakukan prediksi untuk setiap gambar dalam direktori test
predictions = combined_model.predict([test_generator_sehat, test_generator_sakit])
nama_tanaman = ['bok choy', 'daun timun', 'daun tomat', 'kol', 'sawi', 'selada', 'timun', 'tomat']
jenis_penyakit = ['tanpa penyakit', 'dengan penyakit']
results = []

for i, prediction in enumerate(predictions):
    # Ambil nama file gambar dari generator yang sesuai
    if i < len(test_generator_sehat.filenames):
        image_filename = os.path.basename(test_generator_sehat.filenames[i])
    else:
        image_filename = os.path.basename(test_generator_sakit.filenames[i - len(test_generator_sehat.filenames)])

    # Ambil indeks hasil prediksi
    predicted_index = np.argmax(prediction)

    # Ganti hasil prediksi dengan nama buah sesuai indeks
    predicted_label = jenis_penyakit[predicted_index]

    # Simpan pasangan (nama file, hasil prediksi) dalam list
    results.append((image_filename, predicted_label))

# Urutkan berdasarkan nama file
results.sort(key=lambda x: x[0])

# Tampilkan hasil prediksi
for i, (image_filename, predicted_label) in enumerate(results):
    # Dapatkan path lengkap untuk gambar
    if i < len(test_generator_sehat.filenames):
        image_path = os.path.join(sehat_test_dir, image_filename)
    else:
        image_path = os.path.join(sakit_test_dir, image_filename)

    # Tampilkan hasil prediksi
    print(f"Gambar {i + 1} - nama file: {image_filename}, Prediksi: {predicted_label}")


ValueError: ignored

# Ploting Training


# Save Model